In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The forward and backward passes

In [3]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
x_train.shape, x_valid.shape

(torch.Size([50000, 784]), torch.Size([10000, 784]))

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [8]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [9]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [10]:
valid_mean,valid_std = x_valid.mean(),x_valid.std()
valid_mean,valid_std

(tensor(-0.0057), tensor(0.9924))

In [11]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [12]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [13]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [14]:
# num hidden
nh = 50

In [15]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [16]:
m, math.sqrt(m), math.sqrt(nh)

(784, 28.0, 7.0710678118654755)

In [17]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [18]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [19]:
def lin(x, w, b): return x@w + b

In [20]:
t = lin(x_valid, w1, b1)

In [21]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(-0.0662), tensor(1.0053))

In [22]:
def relu(x): return x.clamp_min(0.)

In [23]:
t = relu(lin(x_valid, w1, b1))

In [24]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3656), tensor(0.5689))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [25]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [26]:
w1.mean(),w1.std()

(tensor(9.0288e-05), tensor(0.0505))

In [27]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5417), tensor(0.8250))

In [28]:
# std from scratch
torch.sqrt(((t - t.mean())**2).mean())

tensor(0.8249)

In [29]:
#export
from torch.nn import init

In [30]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [174]:
init.kaiming_normal_??

In [32]:
w1.mean(),w1.std()

(tensor(-8.0204e-05), tensor(0.0505))

In [33]:
t.mean(),t.std()

(tensor(0.6078), tensor(0.8583))

In [34]:
w1.shape

torch.Size([784, 50])

In [35]:
import torch.nn

In [36]:
m,nh

(784, 50)

In [37]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [38]:
torch.nn.Linear.forward??

In [39]:
torch.nn.functional.linear??

In [40]:
torch.nn.Conv2d??

In [175]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [42]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [43]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0433), tensor(0.8100))

In [44]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [45]:
%timeit -n 10 _=model(x_valid)

5.42 ms ± 460 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
torch.Size([x_valid.shape[0],1])

torch.Size([10000, 1])

In [47]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [49]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [50]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [51]:
y_train,y_valid = y_train.float(),y_valid.float()

In [52]:
preds = model(x_train)

In [53]:
preds.shape

torch.Size([50000, 1])

In [54]:
mse(preds, y_train)

tensor(34.8726)

### Gradients and backward pass

    def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [55]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [56]:
# why can you do np.g = ...? actually you can give a tensor any arbitrary attribute...
foo = tensor([1,2])
foo.asfkljafjkdjfk = 1
foo.asfkljafjkdjfk

1

In [57]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [58]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [59]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [137]:
forward_and_backward(x_train, y_train)

In [138]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

-----

In [61]:
inp, targ = x_train, y_train

In [62]:
inp.shape, targ.shape

(torch.Size([50000, 784]), torch.Size([50000]))

In [63]:
l1 = inp @ w1 + b1
l2 = relu(l1)
w1.shape, b1.shape, l1.shape, l2.shape

(torch.Size([784, 50]),
 torch.Size([50]),
 torch.Size([50000, 50]),
 torch.Size([50000, 50]))

In [64]:
out = l2 @ w2 + b2
l2.shape, w2.shape, b2.shape, out.shape

(torch.Size([50000, 50]),
 torch.Size([50, 1]),
 torch.Size([1]),
 torch.Size([50000, 1]))

In [65]:
loss = mse(out, targ)
out.shape, targ.shape, loss.shape

(torch.Size([50000, 1]), torch.Size([50000]), torch.Size([]))

In [67]:
# before calling mse_grad(out, targ), out.g does not exist
out.g

# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# <ipython-input-67-1c83df53566f> in <module>()
# ----> 1 out.g

# AttributeError: 'Tensor' object has no attribute 'g'

AttributeError: 'Tensor' object has no attribute 'g'

In [73]:
# def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()
# 
# def mse_grad(inp, targ): 
#     # grad of loss with respect to output of previous layer
#     inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]
mse_grad(out, targ)

In [76]:
out.shape, out.g.shape

(torch.Size([50000, 1]), torch.Size([50000, 1]))

In [81]:
out.shape[0], targ.shape, out.shape, out.squeeze().shape

(50000, torch.Size([50000]), torch.Size([50000, 1]), torch.Size([50000]))

In [82]:
2. * (out.squeeze() - targ).unsqueeze(-1) / out.shape[0]

tensor([[-2.0881e-04],
        [-7.0711e-05],
        [-1.5671e-04],
        ...,
        [-3.2997e-04],
        [-2.2740e-04],
        [-3.2771e-04]])

In [83]:
out.g

tensor([[-2.0881e-04],
        [-7.0711e-05],
        [-1.5671e-04],
        ...,
        [-3.2997e-04],
        [-2.2740e-04],
        [-3.2771e-04]])

In [85]:
# out = l2 @ w2 + b2
# 
# def lin_grad(inp, out, w, b):
#     # grad of matmul with respect to input
#     inp.g = out.g @ w.t()
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     b.g = out.g.sum(0)
lin_grad(l2, out, w2, b2)

In [112]:
#     inp.g = out.g @ w.t()
out.g.shape, w2.t().shape, (out.g@w2.t()).shape, l2.shape, l2.g.shape

(torch.Size([50000, 1]),
 torch.Size([1, 50]),
 torch.Size([50000, 50]),
 torch.Size([50000, 50]),
 torch.Size([50000, 50]))

In [113]:
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
l2.shape, l2.g.shape, out.shape, out.g.shape

(torch.Size([50000, 50]),
 torch.Size([50000, 50]),
 torch.Size([50000, 1]),
 torch.Size([50000, 1]))

In [114]:
w2.shape, w2.g.shape, 

(torch.Size([50, 1]), torch.Size([50, 1]))

In [115]:
#     b.g = out.g.sum(0)
out.g.shape, out.g.sum(0).shape, b2.shape, b2.g.shape

(torch.Size([50000, 1]), torch.Size([1]), torch.Size([1]), torch.Size([1]))

In [116]:
l2.shape, l2.unsqueeze(-1).shape, out.g.shape, out.g.unsqueeze(1).shape

(torch.Size([50000, 50]),
 torch.Size([50000, 50, 1]),
 torch.Size([50000, 1]),
 torch.Size([50000, 1, 1]))

In [117]:
(l2.unsqueeze(-1) * out.g.unsqueeze(1)).shape

torch.Size([50000, 50, 1])

In [118]:
((l2.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)).shape

torch.Size([50, 1])

In [120]:
#     b.g = out.g.sum(0)

out.g.shape, b2, b2.g

(torch.Size([50000, 1]), tensor([0.]), tensor([-10.2464]))

In [121]:
# def relu(x): return x.clamp_min(0.) - 0.5
# 
# def relu_grad(inp, out):
#     # grad of relu with respect to input activations
#     inp.g = (inp>0).float() * out.g

relu_grad(l1, l2)    

In [124]:
l1.g.shape, l2.g.shape, ((l1>0).float()*l2.g).shape

(torch.Size([50000, 50]),
 torch.Size([50000, 50]),
 torch.Size([50000, 1]),
 torch.Size([50000, 50]))

In [125]:
#     l1 = inp @ w1 + b1
# 
# def lin_grad(inp, out, w, b):
#     # grad of matmul with respect to input
#     inp.g = out.g @ w.t()
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     b.g = out.g.sum(0)
lin_grad(inp, l1, w1, b1)

In [126]:
inp.g.shape, l1.g.shape, (w1.t()).shape

(torch.Size([50000, 784]), torch.Size([50000, 50]), torch.Size([50, 784]))

In [129]:
w1.g.shape, b1.g.shape

(torch.Size([784, 50]), torch.Size([50]))

-----

We cheat a little bit and use PyTorch autograd to check our results.

In [130]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [132]:
w22.grad is None

True

In [133]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [134]:
loss = forward(xt2, y_train)

In [135]:
loss.backward()

In [140]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [141]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [142]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [143]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [144]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [145]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [149]:
display(model.layers)
display(list(reversed(model.layers)))

In [150]:
%time loss = model(x_train, y_train)

CPU times: user 178 ms, sys: 33.8 ms, total: 212 ms
Wall time: 55.7 ms


In [151]:
%time model.backward()

CPU times: user 7.42 s, sys: 14.8 s, total: 22.3 s
Wall time: 37.5 s


In [152]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [153]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [158]:
# old:
# class Relu():
#     def __call__(self, inp):
#         self.inp = inp
#         self.out = inp.clamp_min(0.)-0.5
#         return self.out
    
#     def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [154]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [155]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [156]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [157]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [159]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [160]:
%time loss = model(x_train, y_train)

CPU times: user 185 ms, sys: 729 ms, total: 915 ms
Wall time: 236 ms


In [161]:
%time model.backward()

CPU times: user 372 ms, sys: 89.3 ms, total: 462 ms
Wall time: 181 ms


In [162]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [163]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [164]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [165]:
%time loss = model(x_train, y_train)

CPU times: user 167 ms, sys: 39.3 ms, total: 206 ms
Wall time: 49.8 ms


In [166]:
%time model.backward()

CPU times: user 376 ms, sys: 67 ms, total: 443 ms
Wall time: 161 ms


In [167]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [168]:
#export
from torch import nn

In [169]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [170]:
model = Model(m, nh, 1)

In [171]:
%time loss = model(x_train, y_train)

CPU times: user 167 ms, sys: 41.9 ms, total: 209 ms
Wall time: 53.6 ms


In [172]:
%time loss.backward()

CPU times: user 166 ms, sys: 9.28 ms, total: 175 ms
Wall time: 47.1 ms


## Export

In [173]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
